In [1]:
import pandas as pd
# read data
df = pd.read_csv("stage3.csv")
ownership_df = pd.read_csv("gunownership.csv")

### Gun incidents per year

In [2]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
incidents_per_year = df['year'].value_counts()

# not many incidents from 2013 were recorded (incomplete data) -> removed from set

### Gun incidents per month (per year)

In [3]:
# for each year, counts number of incidents per year
df['month'] = df['date'].dt.month

### Incidents per state

In [4]:
year_per_state = df.groupby(['year', 'state']).size()

In [5]:
incidents_per_state = df.groupby(['state']).size()

In [6]:
# import stuff
from bokeh.plotting import figure
from bokeh.io import output_file, show, save
from bokeh.models import DataRange1d, FactorRange
import pandas as pd

### Some data for plots

In [7]:
# add number of incidents per state
incidents = df.groupby(['state']).size()
incidents = [incident for incident in incidents]
ownership_df['incidents'] = incidents

In [8]:
years_df = pd.DataFrame()
years_df['year'] = ['2014', '2015', '2016', '2017', '2018']
years_df['incidents'] = [number for number in df.groupby('year').size()]

# fill in columns for incidents per yer PER STATE
ownership_df['2014'] = [number for number in year_per_state[:51]]
ownership_df['2015'] = [number for number in year_per_state[51:102]]
ownership_df['2016'] = [number for number in year_per_state[102:153]]
ownership_df['2017'] = [number for number in year_per_state[153:204]]
ownership_df['2018'] = [number for number in year_per_state[204:]]

# incidents per month in the United States
incidents_per_month = [number for number in df.groupby(['year', 'month']).size()]

### Zooming in: analysing cities

In [9]:
df.groupby(['year', 'city_or_county']).size()
df['city_or_county']

0                         Muskegon
1                           Newark
2                           Queens
3                         Brooklyn
4                      Springfield
5                         Columbus
6                         Syracuse
7                        Lexington
8                      Bibb County
9            Prince William County
10                    Philadelphia
11                         Hampton
12                North Charleston
13                       Baltimore
14                      Wilmington
15                  Grand Junction
16                North Charleston
17                     Springfield
18                      Huntsville
19                          Camden
20                     New Orleans
21                       Hurricane
22                      Fort Wayne
23                         Marrero
24                   Pinckneyville
25                           Bronx
26                      Canon City
27                           Bronx
28                  

### Number of registered guns and incidents per state

In [10]:
#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f = figure()
f.x_range=DataRange1d(start=0, end=350000)
f.y_range=DataRange1d(start=0, end=20000)

## Add a Title to the plot (We essentially use css properties here)
# You can find all function of f.title, using dir(f.title)
f.title.text="Registered Guns and Incidents (per State)"
f.title.text_font_size="25px"
f.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f.xaxis.axis_label="Number of Registered Guns"
f.yaxis.axis_label="Number of Incidents"

# Maybe add some different colors for the labels and the digits as well
# Note how we use f.axis, to apply to both the x and y axis.
f.axis.axis_label_text_color="blue"
f.axis.major_label_text_color="red"

#adding glyphs
f.circle(x=ownership_df['guns_registered'], y=ownership_df['incidents'])

#Save and show the figure
save(f)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

### Number of registered guns per capita and incidents per state

In [11]:
#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f2 = figure()

f2.x_range=DataRange1d(start=0, end=250)
f2.y_range=DataRange1d(start=0, end=20000)

## Add a Title to the plot (We essentially use css properties here)
# You can find all function of f.title, using dir(f.title)
f2.title.text="Guns per Capita and Incidents (per State)"
f2.title.text_font_size="25px"
f2.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f2.xaxis.axis_label="Guns per Capita"
f2.yaxis.axis_label="Incidents"

# Maybe add some different colors for the labels and the digits as well
# Note how we use f.axis, to apply to both the x and y axis.
f2.axis.axis_label_text_color="blue"
f2.axis.major_label_text_color="red"

#adding glyphs
f2.circle(x=ownership_df['guns_per_capita'], y=ownership_df['incidents'])

#Save and show the figure
save(f2)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

### Incidents per year (entire country)

In [12]:
#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f3 = figure()

## Add a Title to the plot (We essentially use css properties here)
# You can find all function of f.title, using dir(f.title)
f3.title.text="Gun Incidents in the United States"
f3.title.text_font_size="25px"
f3.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f3.xaxis.axis_label="Year"
f3.yaxis.axis_label="Incidents"

#adding glyphsg
f3.circle(x=years_df['year'], y=years_df['incidents'])

#Save and show the figure
save(f3)

'/home/hughmee/DataProcessing/way-too-many-incidents.html'

### Fatal incidents per year (entire country)

In [13]:
# locate all incidents where people have been killed
deadly_incidents_per_year = [number for number in df.loc[df['n_killed'] > 0].groupby('year').size()]

#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f7 = figure()

## Add a Title to the plot (We essentially use css properties here)
# You can find all function of f.title, using dir(f.title)
f7.title.text="Fatal Gun Incidents in the United States"
f7.title.text_font_size="25px"
f7.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f7.xaxis.axis_label="Year"
f7.yaxis.axis_label="Incidegnts"

#adding glyphsg
f7.circle(x=years_df['year'], y=deadly_incidents_per_year)

#Save and show the figure
save(f7)

'/home/hughmee/DataProcessing/way-too-many-incidents.html'

### Number of people killed each year (entire country)

In [14]:
# get number of people killed each year
per_year = df.groupby('year').sum()
killed_per_year = [killed for killed in per_year['n_killed']]

output_file("way-too-many-incidents.html")
f8 = figure()

# add title
f8.title.text="Gun Deaths in the United States"
f8.title.text_font_size="25px"
f8.title.align="center"

## add some axis information
f8.xaxis.axis_label="Year"
f8.yaxis.axis_label="Killed"

# adding glyphsg
f8.circle(x=years_df['year'], y=killed_per_year)

# save and show the figure
save(f8)

'/home/hughmee/DataProcessing/way-too-many-incidents.html'

### Incidents per month (entire country)

In [15]:
years = ['2014', '2015', '2016', '2017', '2018']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

x_data = [(year, month) for year in years for month in months][:51]
monthly = [number for number in df.groupby(['year', 'month']).size()]

#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f4 = figure(x_range=FactorRange(*x_data), plot_width=800, plot_height=800)

## Add a Title to the plot
f4.title.text="Gun Incidents in the United States (per Month)"
f4.title.text_font_size="25px"
f4.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f4.xaxis.axis_label="Year-Month"
f4.yaxis.axis_label="Number of Incidents"

#adding glyphsg
f4.vbar(x=x_data, top=monthly, width=1, fill_color="#77b7f1")

f4.y_range.start = 0
f4.x_range.range_padding = 0
f4.xaxis.major_label_orientation = 1
#f4.xgrid.grid_line_color = None

#Save and show the figure
save(f4)


'/home/hughmee/DataProcessing/way-too-many-incidents.html'

### Mass Shootings per Month (Entire Country)

In [16]:
mass_shootings = df[df['incident_characteristics'].str.contains("Mass Shooting") == True]

years = ['2014', '2015', '2016', '2017', '2018']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

x_data2 = [(year, month) for year in years for month in months][:51]
monthly2 = [number for number in mass_shootings.groupby(['year', 'month']).size()]

#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f6 = figure(x_range=FactorRange(*x_data2), plot_width=800, plot_height=800)

## Add a Title to the plot
f6.title.text="Mass Shootings in the United States (per Month)"
f6.title.text_font_size="25px"
f6.title.align="center"

## Add some axis information
f6.xaxis.axis_label="Year-Month"
f6.yaxis.axis_label="Number of Incidents"

#adding glyphsg
f6.vbar(x=x_data2, top=monthly2, width=1, fill_color="#f1c177")

f6.y_range.start = 0
f6.x_range.range_padding = 0
f6.xaxis.major_label_orientation = 1
f6.xgrid.grid_line_color = None

#Save and show the figure
save(f6)

'/home/hughmee/DataProcessing/way-too-many-incidents.html'